In [51]:
import pandas as pd
import numpy as np
from datetime import timedelta

In [52]:
import os

def get_csvs(dir:str):
    dirList=os.listdir(dir)
    endResult=[]
    for dire in dirList:
        if(os.path.join(dir,dire).endswith(".csv")):
            endResult.append(dir+"/"+dire)
    return endResult 

In [53]:
allDirs=get_csvs("ScrapedData")

In [82]:
def irregularitiesFinder(df:pd.DataFrame, stockName:str, dic:dict):
    # Calculate percentage change
    df['Pct_Change'] = df['Close'].pct_change()

    # Flag large changes
    threshold = 0.13  # Define a threshold for irregular change, e.g., 10%
    irregularities = df.index[abs(df['Pct_Change']) > threshold].to_list();
    cleanIrregularities = []
    for i, index in enumerate(irregularities):
        if(not ( pd.to_datetime(df.iloc[index,0])-pd.to_datetime(df.iloc[index-1,0])>timedelta(days=3) or (pd.to_datetime(df.iloc[index,0])-pd.to_datetime("2020-03-13 00:00:00+03:00")<timedelta(days=0) and abs(df.iloc[index,6]) < 0.21 ))):
            cleanIrregularities.append(index)


    if(len(cleanIrregularities)==0):
        return

    irregularitiesNP=np.zeros((len(cleanIrregularities),10,7),dtype=object);
    for i, index in enumerate(cleanIrregularities):

        irregularitiesNP[i,:,:]=df.iloc[index-5:index+5,:].to_numpy()

    dic[stockName]=irregularitiesNP

In [65]:
df=pd.read_csv("ScrapedData/AEFES.IS.csv")

In [83]:
stDic={}
for dirs in allDirs:
    df=pd.read_csv(dirs)

    irregularitiesFinder(df,dirs,stDic)
stDic.keys()

dict_keys(['ScrapedData/CCOLA.IS.csv', 'ScrapedData/HALKB.IS.csv', 'ScrapedData/KONTR.IS.csv', 'ScrapedData/ODAS.IS.csv', 'ScrapedData/VAKBN.IS.csv', 'ScrapedData/XBANK.IS.csv', 'ScrapedData/XU030.IS.csv', 'ScrapedData/XU100.IS.csv', 'ScrapedData/XUSIN.IS.csv'])

In [84]:
len(stDic)

9

In [85]:
for stockName in stDic:
    print(stockName)
    print(stDic[stockName].shape)
        

ScrapedData/CCOLA.IS.csv
(1, 10, 7)
ScrapedData/HALKB.IS.csv
(1, 10, 7)
ScrapedData/KONTR.IS.csv
(1, 10, 7)
ScrapedData/ODAS.IS.csv
(1, 10, 7)
ScrapedData/VAKBN.IS.csv
(1, 10, 7)
ScrapedData/XBANK.IS.csv
(1, 10, 7)
ScrapedData/XU030.IS.csv
(1, 10, 7)
ScrapedData/XU100.IS.csv
(1, 10, 7)
ScrapedData/XUSIN.IS.csv
(1, 10, 7)


In [86]:
stDic["ScrapedData/XUSIN.IS.csv"].shape

(1, 10, 7)

In [88]:
mistakesDf = pd.DataFrame(stDic["ScrapedData/XUSIN.IS.csv"][0], columns=df.columns)

In [89]:
mistakesDf

,Date,Open,High,Low,Close,Volume,Pct_Change
0,2020-07-20 00:00:00+03:00,160043.9,161001.0,159708.8,160396.1,1702160.0,0.005099
1,2020-07-21 00:00:00+03:00,161718.2,162101.8,156987.1,158537.1,1673760.0,-0.01159
2,2020-07-22 00:00:00+03:00,159805.6,160457.7,157423.2,158683.5,1675520.0,0.000923
3,2020-07-23 00:00:00+03:00,159592.2,160566.3,159313.4,159612.8,1526120.0,0.005856
4,2020-07-24 00:00:00+03:00,159046.2,160658.3,158622.5,160642.9,1470140.0,0.006454
5,2020-07-27 00:00:00+03:00,1618.5,1631.8,1618.5,1621.7,1368420.0,-0.989905
6,2020-07-28 00:00:00+03:00,1629.4,1630.6,1567.9,1570.7,1194960.0,-0.031448
7,2020-07-29 00:00:00+03:00,1574.1,1587.6,1531.6,1563.4,1209720.0,-0.004648
8,2020-07-30 00:00:00+03:00,1569.1,1569.1,1553.7,1560.1,1268860.0,-0.002111
9,2020-08-04 00:00:00+03:00,1580.0,1587.5,1466.3,1507.2,1228480.0,-0.033908


In [20]:
for stockName in stDic:
    for layer in range(0,stDic[stockName].shape[0]):
        mistakesDf = pd.DataFrame(stDic[stockName][layer], columns=df.columns)
        mistakesDf.to_csv("WeirdData/"+str(layer)+stockName[12:],index=False)
        

In [49]:
def standardize(df,name):
    df['Date'] = pd.to_datetime(df['Date'])
    df['Days_Between'] = df['Date'].diff().dt.days 
    df['Days_Between'] = df['Days_Between'].fillna(1).astype(float)
    
    df['Pct_Change'] = df['Close'].pct_change()
    df["Pct_Change"] = df["Pct_Change"].fillna(0).astype(float)

    #normalize the anomalies
    if name in stDic:
        pass
    #divide the OHLCV by the change percentage


In [50]:
for dirs in allDirs:
    df=pd.read_csv(dirs)

    
    standardize(df,dirs)
    df.to_csv("StandardizedData/"+dirs[12:],index=False)
